In [1]:
import os
import re
import pandas as pd
import numpy as np
import requests

# pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup

In [2]:
def clean_column_names(df):
    clean_names = []
    for col in df.columns.values:
        if 'Unnamed' in col[1]:
            clean_name = col[0]
        else:
            clean_name = '-'.join(col)

        clean_name = re.sub("Año ", "", clean_name)
        clean_names.append(clean_name)

    return clean_names


def clean_precios_promedio(url):
    # Read data
    df = pd.read_excel(url, header=[0,1], skiprows=2)

    # handle multi-index
    df.columns = clean_column_names(df)

    # rename cols
    df = df.rename(columns={"Región":"Region", "Productos seleccionados":"Product", "Unidad de medida":"Unit"})

    # remove region nulls
    df = df[~df.Region.isnull()]

    pct_nulls_cols = df.isnull().sum(axis=0) / df.shape[0]
    cols_to_remove = list(pct_nulls_cols[pct_nulls_cols == 1.0].index)
    df = df.drop(columns=cols_to_remove)

    # Melt (wide to long format)
    id_vars = ['Region', 'Product', 'Unit']
    df = df.melt(id_vars=id_vars)

    # Split 'variable' column into 'year' and 'month' columns
    df[['year', 'month']] = df['variable'].str.split('-', expand=True)

    # Dictionary to map Spanish month names to English
    spanish_to_english = {
        'Enero': 'January', 'Febrero': 'February', 'Marzo': 'March',
        'Abril': 'April', 'Mayo': 'May', 'Junio': 'June',
        'Julio': 'July', 'Agosto': 'August', 'Septiembre': 'September',
        'Octubre': 'October', 'Noviembre': 'November', 'Diciembre': 'December'
    }

    # Convert Spanish month names to English
    df['month'] = df['month'].map(spanish_to_english)

    # Create 'date' column in the desired format
    df['Date'] = pd.to_datetime(df['year'] + '-' + df['month'], format='%Y-%B')

    df = df.rename(columns={'value':'Price'})
    keep_cols = ["Date", 'Region', 'Product', 'Unit', 'Price']
    df = df[keep_cols]    
    df = df[~df.Product.isnull()]
    
    return df


def get_href(soup):
    links = soup.find(class_="contSH hide")
    links = links.find_all('a', class_='a-color2')
    base_url = "https://www.indec.gob.ar"
    return [base_url + link.get("href") for link in links]


In [3]:
url = "https://www.indec.gob.ar/Nivel4/Tema/3/5/31"
payload = ""
headers = {
    "Accept": "text/html, */*; q=0.01",
    "Accept-Language": "es-419,es;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Cookie": "cookiesession1=678A3F6D09D2D493ED2C5D3FD0CBFA6F",
    "Referer": "https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "sec-ch-ua": '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}

response = requests.request("GET", url, data=payload, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")

links_indec = get_href(soup)

for link in links_indec:
    if "sh_ipc_precios_promedio" in link:
        link_sh_ipc_precios_promedio = link
        
print(link_sh_ipc_precios_promedio)

https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_precios_promedio.xls


In [4]:
df = clean_precios_promedio(link_sh_ipc_precios_promedio)
df

,Date,Region,Product,Unit,Price
0,2017-06-01,GBA,Pan francés,kg,38.64
1,2017-06-01,GBA,Harina de trigo común,kg,10.67
2,2017-06-01,GBA,Arroz blanco simple,kg,20.96
3,2017-06-01,GBA,Fideos secos tipo guisero,500 g,19.08
4,2017-06-01,GBA,Carne picada común,kg,72.2
...,...,...,...,...,...
7045,2024-03-01,Patagonia,Papa,kg,1037.45
7046,2024-03-01,Patagonia,Azúcar,kg,1541.32
7047,2024-03-01,Patagonia,Detergente líquido,750 cc,2000.47
7048,2024-03-01,Patagonia,Lavandina,1.000 cc,991.71


In [9]:
output_file = re.sub(".xls", ".csv",
                     link_sh_ipc_precios_promedio.split("/")[-1])

In [10]:
df.to_csv(output_file, index=False)